In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_17542/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_17542/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_16_7,0.910787,0.707327,0.675291,0.781745,0.028337,0.173744,0.227465,0.048141,0.077703,0.168336,0.897246,0.169646,41.127176,89.875262,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_17_2,0.908713,0.707205,0.674470,0.846800,0.028996,0.173816,0.228040,0.043636,0.077259,0.170282,0.894857,0.171607,41.081196,89.829282,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_14_4,0.910875,0.706323,0.677145,0.920527,0.028309,0.174339,0.226166,0.026982,0.078293,0.168253,0.897347,0.169563,41.129135,89.877221,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_16_3,0.910796,0.705364,0.674481,0.784715,0.028334,0.174909,0.228033,0.047486,0.077800,0.168328,0.897256,0.169638,41.127367,89.875453,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_14_8,0.910558,0.704097,0.674877,0.914592,0.028410,0.175661,0.227755,0.028996,0.078085,0.168552,0.896982,0.169864,41.122045,89.870131,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,model_2_0_0,0.029104,-0.057467,-0.130292,0.021230,0.308390,0.627758,0.791793,0.349457,0.281285,0.555329,-0.304642,0.559650,68.352780,162.981417,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1496,model_3_0_3,0.043029,-0.090475,-0.190430,0.233378,0.303967,0.647353,0.833921,0.250307,0.282222,0.551332,-0.543115,0.555622,100.381674,240.890862,"Hidden Size=[12], regularizer=0.02, learning_r..."
1497,model_8_0_1,0.019215,-0.153495,-0.035282,0.222537,0.311531,0.684765,0.725236,0.181300,0.286440,0.558150,-14.815161,0.562493,244.332513,591.304181,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
1498,model_3_0_0,-0.515344,-0.197485,-0.327381,-0.656750,0.481325,0.710879,0.929858,0.540940,0.310652,0.693776,-1.443492,0.699175,99.462424,239.971612,"Hidden Size=[12], regularizer=0.02, learning_r..."
